## Perturbative footprint study

### Imports

In [1]:
# Standard imports
import numpy as np
import shutil
from ruamel.yaml import YAML
import os

# Cern modules
import fillingpatterns as fp
#import xmask as xm
#import xtrack as xt

# Improve style
import seaborn as sns
sns.set_theme(style="whitegrid")


### Generate a machine with xmask from the appropriate configuration file

#### First create a proper configuration file

In [2]:
# Copy default configuration file for xmask, and target configuration file for pymask
shutil.copyfile('../modules/xmask/examples/hllhc15_collision/config.yaml', 'config_xm.yaml')
shutil.copyfile('/afs/cern.ch/work/c/cdroin/private/DA_study/master_study/opt_flathv_75_1500_withBB_chroma15_1p4_all_bunches/madx_001/config.yaml', 'config_pm.yaml')


# Read and mutate xmask config file
with open("config_xm.yaml", "r") as fid:
    config_xm_str = fid.read()

with open("config_pm.yaml", "r") as fid:
    config_pm_str = fid.read()
    
yaml = YAML()
config_pm = yaml.load(config_pm_str)
config_xm = yaml.load(config_xm_str)

# Hardcode optics configuration files as they can't be ported directly
config_xm['config_mad']['links']['acc-models-lhc'] = '/afs/cern.ch/eng/lhc/optics/HLLHCV1.5'
config_xm['config_mad']['optics_file'] = 'acc-models-lhc/flatcc/opt_flathv_75_180_1500_thin.madx'
config_xm['config_knobs_and_tuning']['closed_orbit_correction']['lhcb1'] = '../modules/tools/corr_co_lhcb1.json'
config_xm['config_knobs_and_tuning']['closed_orbit_correction']['lhcb2'] = '../modules/tools/corr_co_lhcb2.json'

# Mutate optics and check config
config_xm['config_mad']['enable_imperfections'] = config_pm['enable_imperfections']
config_xm['config_mad']['enable_knob_synthesis'] = config_pm['enable_knob_synthesis']
config_xm['config_mad']['ver_hllhc_optics'] = float(config_pm['optics_version'])
config_xm['config_mad']['pars_for_imperfections'] = config_pm['pars_for_imperfections']

# Mutate beam config
config_xm['config_mad']['beam_config']['lhcb1']['beam_energy_tot'] = config_pm['beam_energy_tot']
config_xm['config_mad']['beam_config']['lhcb2']['beam_energy_tot'] = config_pm['beam_energy_tot']

# Mutate beambeam parameters
config_xm['config_beambeam']['nemitt_x'] = config_pm['beam_norm_emit_x']*1e-6
config_xm['config_beambeam']['nemitt_y'] = config_pm['beam_norm_emit_y']*1e-6
config_xm['config_beambeam']['sigma_z'] = config_pm['beam_sigt']
config_xm['config_beambeam']['num_particles_per_bunch'] = config_pm['beam_npart']
config_xm['config_beambeam']['num_particles_per_bunch'] = config_pm['beam_npart']
for i, ip in enumerate(['ip1', 'ip2', 'ip5', 'ip8']):
    config_xm['config_beambeam']['num_long_range_encounters_per_side'][ip] = config_pm['beambeam_config']['numberOfLRPerIRSide'][i]
config_xm['config_beambeam']['num_slices_head_on'] = config_pm['beambeam_config']['numberOfHOSlices']
config_xm['config_beambeam']['bunch_spacing_buckets'] = config_pm['beambeam_config']['bunch_spacing_buckets']

# Mutate tunes and chromaticities (assume same for both beams, since not settable in pymask)
config_xm['config_knobs_and_tuning']['qx']['lhcb1'] = config_pm['qx0']
config_xm['config_knobs_and_tuning']['qx']['lhcb2'] = config_pm['qx0']
config_xm['config_knobs_and_tuning']['qy']['lhcb1'] = config_pm['qy0']
config_xm['config_knobs_and_tuning']['qy']['lhcb2'] = config_pm['qy0']
config_xm['config_knobs_and_tuning']['dqx']['lhcb1'] = config_pm['chromaticity_x']
config_xm['config_knobs_and_tuning']['dqx']['lhcb2'] = config_pm['chromaticity_x']
config_xm['config_knobs_and_tuning']['dqy']['lhcb1'] = config_pm['chromaticity_y']
config_xm['config_knobs_and_tuning']['dqy']['lhcb2'] = config_pm['chromaticity_y']

# Mutate knobs
for knob in config_pm['knob_settings']:
    if knob in config_xm['config_knobs_and_tuning']['knob_settings']:
        config_xm['config_knobs_and_tuning']['knob_settings'][knob] = config_pm['knob_settings'][knob]
    else:
        print(f"Knob {knob} not found in xmask config file. Beind added now.")
        config_xm['config_knobs_and_tuning']['knob_settings'][knob] = config_pm['knob_settings'][knob]

# Mutate RF voltage and octupole current
config_xm['config_knobs_and_tuning']['knob_settings']['vrf400'] = config_pm['vrf_total']
config_xm['config_knobs_and_tuning']['knob_settings']['i_oct_b1'] = config_pm['oct_current']
config_xm['config_knobs_and_tuning']['knob_settings']['i_oct_b2'] = config_pm['oct_current']


# Artificially implement end of luminosity levelling
config_xm['config_knobs_and_tuning']['knob_settings']['on_sep8'] = 0.0
config_xm['config_knobs_and_tuning']['knob_settings']['on_sep8h'] = -0.01745641501719127
config_xm['config_knobs_and_tuning']['knob_settings']['on_sep8v'] = 0.01371863979152592
config_xm['config_knobs_and_tuning']['knob_settings']['on_x8'] = 0.0
config_xm['config_knobs_and_tuning']['knob_settings']['on_x8h'] = 0.0
config_xm['config_knobs_and_tuning']['knob_settings']['on_x8v'] = 170.0
config_xm['config_knobs_and_tuning']['knob_settings']['on_sep2'] = 0.1443593672910653
 

# ! WARNING. AT THIS POINT:
# ! - Coupling is missing
# ! - bunch-by-bunch and filling schemes are missing


# Write config file
with open("config_xm.yaml", "w") as fid:
    yaml.dump(config_xm, fid)


Knob on_x8h not found in xmask config file. Beind added now.
Knob on_x8v not found in xmask config file. Beind added now.
Knob on_sep8h not found in xmask config file. Beind added now.
Knob on_sep8v not found in xmask config file. Beind added now.


#### Build machine

In [3]:
os.system("python build_machine.py")

rm: cannot remove 'acc-models-lhc': No such file or directory
rm: cannot remove 'temp': No such file or directory



  ++++++++++++++++++++++++++++++++++++++++++++
  +     MAD-X 5.08.01  (64 bit, Linux)       +
  + Support: mad@cern.ch, http://cern.ch/mad +
  + Release   date: 2022.02.25               +
  + Execution date: 2023.04.19 15:46:58      +
  ++++++++++++++++++++++++++++++++++++++++++++

  ++++++++++++++++++++++++++++++++++++++++++++
  +     MAD-X 5.08.01  (64 bit, Linux)       +
  + Support: mad@cern.ch, http://cern.ch/mad +
  + Release   date: 2022.02.25               +
  + Execution date: 2023.04.19 15:46:59      +
  ++++++++++++++++++++++++++++++++++++++++++++
! 2011 Stephane Fartoukh, macros for building ATS optics

!   From 2011/11/10 /afs/cern.ch/eng/lhc/optics/ATS_V6.503/toolkit/macro.max

!   select(NIR,sec1,sec2,BIM)

!   backtrack_twiss

!   SELECT_BACK(NIR,sec1,sec2,BIM)

!   SELECT1(NIR,BIM)

!   selectIR15(NIR,SEC1,SEC2,BIM):

!       put in b/eirNIRBIM DeltaPhi to MS14/MS15 such that the phase is

!       actually between the MS

!       1.25: xlb1, yrb1, ylb2, xrb2

!       

rm: cannot remove 'errors': No such file or directory


installing mqxfa.a1r5..FL mqxfa.a1r5..FR
installing mqxfa.b1r5..FL mqxfa.b1r5..FR
installing mqxfb.a2r5..FL mqxfb.a2r5..FR
installing mqxfb.b2r5..FL mqxfb.b2r5..FR
installing mqxfa.a3r5..FL mqxfa.a3r5..FR
installing mqxfa.b3r5..FL mqxfa.b3r5..FR
installing mqxfa.a1l5..FL mqxfa.a1l5..FR
installing mqxfa.b1l5..FL mqxfa.b1l5..FR
installing mqxfb.a2l5..FL mqxfb.a2l5..FR
installing mqxfb.b2l5..FL mqxfb.b2l5..FR
installing mqxfa.a3l5..FL mqxfa.a3l5..FR
installing mqxfa.b3l5..FL mqxfa.b3l5..FR
installing mqxfa.a1r1..FL mqxfa.a1r1..FR
installing mqxfa.b1r1..FL mqxfa.b1r1..FR
installing mqxfb.a2r1..FL mqxfb.a2r1..FR
installing mqxfb.b2r1..FL mqxfb.b2r1..FR
installing mqxfa.a3r1..FL mqxfa.a3r1..FR
installing mqxfa.b3r1..FL mqxfa.b3r1..FR
installing mqxfa.a1l1..FL mqxfa.a1l1..FR
installing mqxfa.b1l1..FL mqxfa.b1l1..FR
installing mqxfb.a2l1..FL mqxfb.a2l1..FR
installing mqxfb.b2l1..FL mqxfb.b2l1..FR
installing mqxfa.a3l1..FL mqxfa.a3l1..FR
installing mqxfa.b3l1..FL mqxfa.b3l1..FR
installing mcbxf

59874ef1b527467ab7c843d0468310a4.c:4280: warning: "_GNU_SOURCE" redefined
 4280 | #  define _GNU_SOURCE // enable GNU libc NAN extension if possible
      | 
In file included from 59874ef1b527467ab7c843d0468310a4.c:50:
/afs/cern.ch/work/c/cdroin/private/xmask_DA_study/miniconda/include/python3.10/pyconfig.h:1621: note: this is the location of the previous definition
 1621 | #define _GNU_SOURCE 1
      | 


...

e22885f099ac41798bfb239c83d3d11e.c:5537: warning: "_GNU_SOURCE" redefined
 5537 | #  define _GNU_SOURCE // enable GNU libc NAN extension if possible
      | 
In file included from e22885f099ac41798bfb239c83d3d11e.c:50:
/afs/cern.ch/work/c/cdroin/private/xmask_DA_study/miniconda/include/python3.10/pyconfig.h:1621: note: this is the location of the previous definition
 1621 | #define _GNU_SOURCE 1
      | 


Done compiling ContextCpu kernels.
Compiling ContextCpu kernels...
Done compiling ContextCpu kernels.


Traceback (most recent call last):
  File "/afs/cern.ch/work/c/cdroin/private/xmask_DA_study/footprint_study/build_machine.py", line 125, in <module>
    collider.to_json("output/collider_tuned_bb_on.json")
  File "/afs/cern.ch/work/c/cdroin/private/xmask_DA_study/miniconda/lib/python3.10/site-packages/xtrack/multiline/multiline.py", line 134, in to_json
    json.dump(self.to_dict(**kwargs), fid, cls=xo.JEncoder)
  File "/afs/cern.ch/work/c/cdroin/private/xmask_DA_study/miniconda/lib/python3.10/site-packages/xtrack/multiline/multiline.py", line 57, in to_dict
    dct['_var_manager'] = self._var_sharing.manager.dump()
  File "/afs/cern.ch/work/c/cdroin/private/xmask_DA_study/miniconda/lib/python3.10/site-packages/xdeps/tasks.py", line 356, in dump
    for t in self.find_tasks(self.rdeps)
  File "/afs/cern.ch/work/c/cdroin/private/xmask_DA_study/miniconda/lib/python3.10/site-packages/xdeps/tasks.py", line 239, in find_tasks
    return [self.tasks[taskid] for taskid in self.find_taskids(s


  Number of warnings: 116
116 in C and 0 in Fortran

  ++++++++++++++++++++++++++++++++++++++++++++
  +          MAD-X finished normally         +
  ++++++++++++++++++++++++++++++++++++++++++++

  Number of warnings: 60
60 in C and 0 in Fortran

  ++++++++++++++++++++++++++++++++++++++++++++
  +          MAD-X finished normally         +
  ++++++++++++++++++++++++++++++++++++++++++++


256

### Correct bbb schedule

In [4]:
# Get BB schedule
fname = "filling_scheme/8b4e_1972b_1960_1178_1886_224bpi_12inj_800ns_bs200ns.json"
patt = fp.FillingPattern.from_json(fname)

# Compute bb schedule
patt.compute_beam_beam_schedule(n_lr_per_side=16)
bbs = patt.b1.bb_schedule

# Write bb schedule
bbs.to_csv("filling_scheme/8b4e_1972b_1960_1178_1886_224bpi_12inj_800ns_bs200ns_bbs.csv", index_label="bunch number")

# Get list of bunches

NameError: name 'fp' is not defined

## Plot footprint

Coupling correction due to the lattice (```delta_cmr``` and ```delta_cmi``` in pymask) are ignored as they're not very relevant for the coupling study.